### Routing (filter_unique_query)

![rag flow image](routing-flow.png "RAG FLOW")

website - https://chaidocs.vercel.app/youtube/getting-started

### Logical Routing (filter_unique_query)

![rag flow image](logical-router.png "RAG FLOW")